# Target Mask Generation & Visualization

This notebook contains the **standalone code** to generate and visualize the Garment-Independent Target Mask (Target Guidance Map) for the virtual try-on Flow Estimation Module (FEM).

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
import os, cv2, numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

def get_target_mask(parse_path, densepose_path):
    """
    STANDALONE LOGIC:
    Generates a neutral target mask using DensePose and SCHP Parsing.
    """
    # Load maps
    parse = np.array(Image.open(parse_path).convert("L"))
    dp_segm = np.array(Image.open(densepose_path).convert("L"))

    # 1. Identify Target Body Surface (DensePose Labels)
    # Torso: 1, 2 | Arms: 11, 12, 13, 14
    guidance_mask = np.isin(dp_segm, [1, 2, 11, 12, 13, 14]).astype(np.uint8)

    # 2. Refine Mask (Morphology)
    kernel = np.ones((5,5), np.uint8)
    # Closing fills gaps between folded arms and torso
    neutral_target = cv2.morphologyEx(guidance_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    # Dilation for full body coverage
    neutral_target = cv2.dilate(neutral_target, kernel, iterations=2)

    # 3. Preservation Check (Protect Face/Hair)
    # LIP Labels: Hat: 1, Hair: 2, Face: 13
    preserve_mask = np.isin(parse, [1, 2, 13]).astype(np.uint8)
    neutral_target[preserve_mask == 1] = 0

    # 4. Construct Final Mask
    # Start with original parsing but clear garment area (5, 6, 7)
    res_mask = parse.copy()
    res_mask[np.isin(parse, [5, 6, 7])] = 0
    
    # Fill the identified neutral body surface with Label 5 (Upper Clothes)
    res_mask[neutral_target == 1] = 5

    return res_mask

## Visualization
Select an ID and display the input maps alongside the generated target mask.

In [8]:
# Set paths based on your environment (Local or Google Drive)
DATA_ROOT = Path("d:/Virtual try on/virtual-tryon/data/viton_hd/test")
if not DATA_ROOT.exists():
    DATA_ROOT = Path("/content/drive/MyDrive/virtual_tryon_project/data/viton_hd/train")

person_id = "00005_00"

img_path = DATA_ROOT / 'image' / f"{person_id}.jpg"
parse_path = DATA_ROOT / 'image-parse-v3' / f"{person_id}.png"
dp_path = DATA_ROOT / 'image-densepose' / f"{person_id}.png"

if all(p.exists() for p in [img_path, parse_path, dp_path]):
    # Generate Mask
    target_mask = get_target_mask(parse_path, dp_path)
    
    # Create Visual Comparison
    fig, ax = plt.subplots(1, 4, figsize=(24, 12))
    
    ax[0].imshow(Image.open(img_path))
    ax[0].set_title("1. Original Image")
    
    ax[1].imshow(Image.open(parse_path), cmap='nipy_spectral')
    ax[1].set_title("2. SCHP Parsing (Original)")
    
    ax[2].imshow(Image.open(dp_path), cmap='viridis')
    ax[2].set_title("3. DensePose (Body Surface)")
    
    ax[3].imshow(target_mask, cmap='nipy_spectral')
    ax[3].set_title("4. GENERATED TARGET MASK\n(Garment-Independent)")
    
    for a in ax: a.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("❌ Files not found. Please check paths in cell above.")

❌ Files not found. Please check paths in cell above.
